# Recommending Music to train to using an existing Spotify playlist

*Code by Sean Brockway*

### Necessary imports for Spotify and MongoDB Connections

In [2]:
import requests
import json
import pymongo
import spotipy
import spotipy.util as util 
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from spotipy.oauth2 import SpotifyClientCredentials
from pymongo import MongoClient
import urllib.parse

### Setting up connections for MongoDB and Spotify API

In [3]:
client_credentials_manager = SpotifyClientCredentials()
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

conn = MongoClient("mongodb://localhost:27017/")
db = conn['gym-music-database']
test_playlist_col = db['example_playlist']
playlist_songs = test_playlist_col.find({})

**Verifying our data is correct, this is an example of a song in our playlist**

In [4]:
print(playlist_songs[0])

{'_id': ObjectId('5e83559b93d27f4463566fd0'), 'track': 'Hit It', 'track_id': '7JXdE4b0hpNSlZmPyopgii', 'artist_name': ['American Authors'], 'artist_id': ['0MlOPi3zIDMVrfA9R04Fe3'], 'genres': ['indie poptimism', 'modern alternative rock', 'modern rock', 'neo mellow', 'pop rock'], 'danceability': 0.645, 'energy': 0.919, 'key': 7, 'loudness': -3.669, 'mode': 1, 'speechiness': 0.0853, 'acousticness': 0.0025, 'instrumentalness': 0, 'liveness': 0.527, 'valence': 0.77, 'tempo': 120.057, 'duration_ms': 207453, 'time_signature': 4}


**Next we need to make a list of every song's genres**

In [5]:
genre_list = []
for song in playlist_songs:
    
#genre_list

**Next we need to find the most common occuring genres to base our recommendations on**

In [6]:
from collections import Counter
most_common_genres = [genre for genre, genre_count in Counter(genre_list).most_common(8)]
print(most_common_genres)

['modern rock', 'indie pop', 'indietronica', 'alternative dance', 'indie rock', 'new rave', 'rock', 'shimmer pop']


To get ideas for songs to recommend, the search end point and recommendations end point in the API can be used. For the search end point, spaces will not work, so it is best to use the "replace" method to change spaces to "%20"

The recommendations end point in the Spotify API can also be used to generate a list of song recommendations, this does not require any text to be replaced. 

In [7]:
search_genres = []
for genre in most_common_genres:
    search = genre.replace(" ", "%20")
    search_genres.append(search)
print(search_genres)

['modern%20rock', 'indie%20pop', 'indietronica', 'alternative%20dance', 'indie%20rock', 'new%20rave', 'rock', 'shimmer%20pop']


#### Now find recommendations using the 5 most common genres. 
It is worth noting that the recommendations end point has lots of different paramaters and kwargs, and bespoke recommendations can be made using the detailed spotify audio features. For now though, we will just use genre.

Firstly we need to get all the available genre seeds used to generate recommendations

In [8]:
seeds = sp.recommendation_genre_seeds()
print(seeds['genres'])

['acoustic', 'afrobeat', 'alt-rock', 'alternative', 'ambient', 'anime', 'black-metal', 'bluegrass', 'blues', 'bossanova', 'brazil', 'breakbeat', 'british', 'cantopop', 'chicago-house', 'children', 'chill', 'classical', 'club', 'comedy', 'country', 'dance', 'dancehall', 'death-metal', 'deep-house', 'detroit-techno', 'disco', 'disney', 'drum-and-bass', 'dub', 'dubstep', 'edm', 'electro', 'electronic', 'emo', 'folk', 'forro', 'french', 'funk', 'garage', 'german', 'gospel', 'goth', 'grindcore', 'groove', 'grunge', 'guitar', 'happy', 'hard-rock', 'hardcore', 'hardstyle', 'heavy-metal', 'hip-hop', 'holidays', 'honky-tonk', 'house', 'idm', 'indian', 'indie', 'indie-pop', 'industrial', 'iranian', 'j-dance', 'j-idol', 'j-pop', 'j-rock', 'jazz', 'k-pop', 'kids', 'latin', 'latino', 'malay', 'mandopop', 'metal', 'metal-misc', 'metalcore', 'minimal-techno', 'movies', 'mpb', 'new-age', 'new-release', 'opera', 'pagode', 'party', 'philippines-opm', 'piano', 'pop', 'pop-film', 'post-dubstep', 'power-po

Now we find the genres in our playlist that can be used as a genre seed

In [9]:
list_A = most_common_genres
list_B = seeds['genres']
jVal = "|".join(list_A)

matches = [i for i in list_B if i in jVal ]
print(matches)

['alternative', 'dance', 'indie', 'pop', 'rock']


In [10]:
query_string = ','.join(matches)
print(query_string)

alternative,dance,indie,pop,rock


In [25]:
recommended_tracks = sp.recommendations(seed_genres = matches, limit=20)
#print(recommended_tracks['tracks'])

Here are the recommended tracks from Spotify, now we need to make sure there are no duplicate tracks that exist in the existing playlist, then we need to get the audio features for each track and prepare them for our classifier. 

In [12]:
for track  in recommended_tracks['tracks']:
    if any(track['id'] in songs for songs in playlist_songs):
        print('match found')
    else:
        print('no match')

no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match


No matches have been found, now we can begin getting the audio features for each song and building a data frame for our model to make predictions on. 

In [13]:
dataframe_columns = ['track', 'track_id', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature']
song_list = []
for track in recommended_tracks['tracks']:
    features = sp.audio_features(track['id'])
    if (features[0] is not None):
                song_record = { 
                    "track": track['name'], 
                    "track_id": track['id'],
                    "danceability" : features[0]['danceability'],
                    "energy" : features[0]['energy'],
                    "key" : features[0]['key'],
                    "loudness" : features[0]['loudness'],
                    "mode" : features[0]['mode'],
                    "speechiness" : features[0]['speechiness'],
                    "acousticness" : features[0]['acousticness'],
                    "instrumentalness" : features[0]['instrumentalness'],
                    "liveness" : features[0]['liveness'],
                    "valence" : features[0]['valence'],
                    "tempo" : features[0]['tempo'],
                    "duration_ms" : features[0]['duration_ms'],
                    "time_signature" : features[0]['time_signature']
                } 
                song_list.append(song_record)
#print(song_list)
dfRecommendations = pd.DataFrame.from_dict(song_list)
dfRecommendations

,track,track_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Master Of None,2zkTK6l0Du0nEgwPHGGT0p,0.497,0.347,5,-15.256,1,0.0260,0.620000,0.529000,0.0942,0.2120,86.486,199413,4
1,Shining (feat. Beyoncé & Jay-Z),7Fa5UNizycSms5jP3SQD3F,0.768,0.807,2,-5.508,0,0.1060,0.053000,0.000003,0.2140,0.5750,112.001,284000,4
2,You Gotta Not,0B1QTVABE86QcHjXdPfeej,0.610,0.803,1,-4.523,1,0.0350,0.010700,0.000029,0.0386,0.4380,89.997,191027,4
3,The Way Life Goes (feat. Oh Wonder),2eAZfqOm4EnOF9VvN50Tyc,0.703,0.757,1,-3.612,1,0.0500,0.617000,0.000000,0.1060,0.6960,82.008,221747,4
4,Renegades Of Funk,5YBVDvTSSSiqv7KZDeUlXA,0.651,0.908,2,-4.019,1,0.0577,0.001840,0.000579,0.3840,0.7690,123.851,274160,4
5,Word Up!,4D05EjPFWFx3BEQNeB7Vct,0.580,0.939,6,-4.425,0,0.0369,0.000027,0.064400,0.3490,0.8040,116.042,172667,4
6,Multi-Family Garage Sale (Bargain-Bin Mix),0hia3XiJkFZqBkToRyX83g,0.837,0.683,2,-13.546,1,0.0979,0.447000,0.680000,0.1790,0.7850,102.056,255133,4
7,Misery Business,6SpLc7EXZIPpy0sVko0aoU,0.517,0.906,1,-3.677,1,0.0735,0.002720,0.000009,0.1130,0.7310,172.977,211520,4
8,8 (circle),47IklCMgkgWvI4jpkdrop0,0.404,0.408,9,-10.770,1,0.0349,0.911000,0.051200,0.1170,0.0723,125.004,309013,4
9,Magnolia,1e1JKLEDKP7hEQzJfNAgPl,0.791,0.581,11,-7.323,0,0.2860,0.011300,0.000000,0.3500,0.4380,162.990,181812,4


Now we have a data frame which we can use to predict if a song is gym suitable or not. Before this happens, we need to train our model. We will do this with 10 thousands records of training data using the random forest algorthim. 

In [14]:
training_col = db['medium_test_songlist']
training_songs = training_col.find({})
dfTraining =  pd.DataFrame(list(training_songs))
dfTraining

,_id,track,track_id,artist_name,artist_id,from_playlist,genres,danceability,energy,key,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,workout_suitable
0,5e7e4fa0b96e22bd0ee61d99,'Till I Collapse,4xkOaSrkexMciUUogZKVTS,"[Eminem, Nate Dogg]","[7dGJo4pcD2V6oG8kP0tJRR, 1Oa0bMld0A3u5OTYfMzp5h]","Gym Motivation : Workout Motivation, Training ...","[detroit hip hop, g funk, hip hop, rap, g funk...",0.548,0.8470,1,...,1,0.1860,0.06220,0.000000,0.0816,0.1000,171.447,297787,4,1
1,5e7e4fa1b96e22bd0ee61d9a,Numb / Encore,5sNESr6pQfIhL3krM8CtZn,"[JAY-Z, Linkin Park]","[3nFkdlSjzX9mRTtwJOzDYB, 6XyY86QOPPrYVGvF9ch6wz]","Gym Motivation : Workout Motivation, Training ...","[east coast hip hop, hip hop, pop rap, rap, al...",0.687,0.7930,2,...,1,0.1660,0.06030,0.000000,0.5820,0.7510,107.045,205733,4,1
2,5e7e4fa1b96e22bd0ee61d9b,Remember the Name (feat. Styles of Beyond),6ndmKwWqMozN2tcZqzCX4K,"[Fort Minor, Styles Of Beyond]","[7dWYWUbO68rXJOcyA7SpJk, 5bf6yYgHODBW5EreBZshpX]","Gym Motivation : Workout Motivation, Training ...","[rap rock, cali rap, rap rock]",0.688,0.8350,8,...,1,0.0911,0.05830,0.000003,0.0795,0.8800,84.858,230493,4,1
3,5e7e4fa1b96e22bd0ee61d9c,In Da Club,4RY96Asd9IefaL3X4LOLZ8,[50 Cent],[3q7HBObVc0L8jNeTe5Gofh],"Gym Motivation : Workout Motivation, Training ...","[east coast hip hop, gangster rap, hip hop, po...",0.902,0.7200,6,...,0,0.3470,0.26000,0.000000,0.0749,0.8050,90.059,193467,4,1
4,5e7e4fa1b96e22bd0ee61d9d,Stronger,4fzsfWzRhPawzqhX8Qt9F3,[Kanye West],[5K4W6rqBFWDnAN6FQUkS6x],"Gym Motivation : Workout Motivation, Training ...","[chicago rap, rap]",0.617,0.7170,10,...,0,0.1530,0.00564,0.000000,0.4080,0.4900,103.992,311867,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12678,5e7e63f38aa9abc22c18141a,Hallelujah,0uCGsNZqjHgiYO4BwN6Cjw,[Ben Laver],[3lTGvG2QAIoGMp7BKeH4C0],Calming Instrumental Covers,[],0.428,0.0415,7,...,0,0.0476,0.98500,0.942000,0.1060,0.2810,71.353,165882,4,0
12679,5e7e63f48aa9abc22c18141b,Close To Me,0kPLGFA5BjDGVkDbu4bLzE,[Mia Carrera],[00zoOzjXViIF4Fs8XQSK9m],Calming Instrumental Covers,"[calming instrumental, piano cover]",0.636,0.2410,4,...,1,0.0494,0.99000,0.902000,0.1510,0.1210,102.094,203587,4,0
12680,5e7e63f48aa9abc22c18141c,Love Me Like You Do,1nyxHNRjeyTlJQwsLMH7C0,[Anna Kitkowska],[45Miu8OyhYvkkQBXhn0MfQ],Calming Instrumental Covers,"[calming instrumental, chill guitar]",0.635,0.2310,11,...,0,0.0419,0.93900,0.947000,0.0925,0.3210,144.229,198208,4,0
12681,5e7e63f48aa9abc22c18141d,Fix You,2ZzqC4NSi374IufPYvGMP6,[Music Lab Collective],[1ylcY77FWeSVQKh5et1VGp],Calming Instrumental Covers,[classify],0.667,0.0498,3,...,1,0.0673,0.99100,0.890000,0.0818,0.0436,125.070,332427,4,0


Here is the dataframe we will use to train our classifier, next we define the features we want the classifier to learn and the feature we want to predict. These are named X and y respectively.

In [15]:
feature_cols = ['acousticness', 'danceability', 'duration_ms', 'energy', 'instrumentalness', 'key', 'liveness', 'mode', 'speechiness', 'tempo', 'time_signature']
X = dfTraining.loc[:, feature_cols]
y = dfTraining.workout_suitable

In [16]:
X

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,mode,speechiness,tempo,time_signature
0,0.06220,0.548,297787,0.8470,0.000000,1,0.0816,1,0.1860,171.447,4
1,0.06030,0.687,205733,0.7930,0.000000,2,0.5820,1,0.1660,107.045,4
2,0.05830,0.688,230493,0.8350,0.000003,8,0.0795,1,0.0911,84.858,4
3,0.26000,0.902,193467,0.7200,0.000000,6,0.0749,0,0.3470,90.059,4
4,0.00564,0.617,311867,0.7170,0.000000,10,0.4080,0,0.1530,103.992,4
...,...,...,...,...,...,...,...,...,...,...,...
12678,0.98500,0.428,165882,0.0415,0.942000,7,0.1060,0,0.0476,71.353,4
12679,0.99000,0.636,203587,0.2410,0.902000,4,0.1510,1,0.0494,102.094,4
12680,0.93900,0.635,198208,0.2310,0.947000,11,0.0925,0,0.0419,144.229,4
12681,0.99100,0.667,332427,0.0498,0.890000,3,0.0818,1,0.0673,125.070,4


In [17]:
y

0        1
1        1
2        1
3        1
4        1
        ..
12678    0
12679    0
12680    0
12681    0
12682    0
Name: workout_suitable, Length: 12683, dtype: int64

## Prediction

#### Constructing a prediction model using X and y

In [18]:
rf = RandomForestClassifier(n_estimators=40)
workout_classifier = rf.fit(X, y)
workout_classifier

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=40,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

#### Now we prepare the song recommendations to be tested 

In [19]:
X_predict = dfRecommendations.loc[:, feature_cols]
X_predict.shape

(20, 11)

### Using the classifer to predict the recommendations (X_predict)

In [20]:
new_prediction_class = workout_classifier.predict(X_predict)
new_prediction_class

array([0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1])

 This is the array of predictions, we can build a new dataframe to display this information more clearly

In [21]:
dfClassifiedRecommendations = pd.DataFrame({
    'Track Name': dfRecommendations.track,
    'Track_id': dfRecommendations.track_id,
    'Prediction': new_prediction_class
})
dfClassifiedRecommendations

,Track Name,Track_id,Prediction
0,Master Of None,2zkTK6l0Du0nEgwPHGGT0p,0
1,Shining (feat. Beyoncé & Jay-Z),7Fa5UNizycSms5jP3SQD3F,1
2,You Gotta Not,0B1QTVABE86QcHjXdPfeej,1
3,The Way Life Goes (feat. Oh Wonder),2eAZfqOm4EnOF9VvN50Tyc,1
4,Renegades Of Funk,5YBVDvTSSSiqv7KZDeUlXA,1
5,Word Up!,4D05EjPFWFx3BEQNeB7Vct,1
6,Multi-Family Garage Sale (Bargain-Bin Mix),0hia3XiJkFZqBkToRyX83g,0
7,Misery Business,6SpLc7EXZIPpy0sVko0aoU,1
8,8 (circle),47IklCMgkgWvI4jpkdrop0,0
9,Magnolia,1e1JKLEDKP7hEQzJfNAgPl,1


Here we have a list of songs with the classifiers prediction of if it is suitable for working out to or not. 
Now all that is left to do is display only the songs predicted as suitable for our hypothetical user.

In [22]:
dict1 = dfClassifiedRecommendations.to_dict('records')
for track in dict1:
    print(track)

{'Track Name': 'Master Of None', 'Track_id': '2zkTK6l0Du0nEgwPHGGT0p', 'Prediction': 0}
{'Track Name': 'Shining (feat. Beyoncé & Jay-Z)', 'Track_id': '7Fa5UNizycSms5jP3SQD3F', 'Prediction': 1}
{'Track Name': 'You Gotta Not', 'Track_id': '0B1QTVABE86QcHjXdPfeej', 'Prediction': 1}
{'Track Name': 'The Way Life Goes (feat. Oh Wonder)', 'Track_id': '2eAZfqOm4EnOF9VvN50Tyc', 'Prediction': 1}
{'Track Name': 'Renegades Of Funk', 'Track_id': '5YBVDvTSSSiqv7KZDeUlXA', 'Prediction': 1}
{'Track Name': 'Word Up!', 'Track_id': '4D05EjPFWFx3BEQNeB7Vct', 'Prediction': 1}
{'Track Name': 'Multi-Family Garage Sale (Bargain-Bin Mix)', 'Track_id': '0hia3XiJkFZqBkToRyX83g', 'Prediction': 0}
{'Track Name': 'Misery Business', 'Track_id': '6SpLc7EXZIPpy0sVko0aoU', 'Prediction': 1}
{'Track Name': '8 (circle)', 'Track_id': '47IklCMgkgWvI4jpkdrop0', 'Prediction': 0}
{'Track Name': 'Magnolia', 'Track_id': '1e1JKLEDKP7hEQzJfNAgPl', 'Prediction': 1}
{'Track Name': 'Over and Over Again', 'Track_id': '3nhGcnuGvsLL8Otj

In [23]:
dictRecommend = []
for track in dict1: 
    if(track['Prediction'] == 1):
        dictRecommend.append(track)
for track in dictRecommend:
    print(track)

{'Track Name': 'Shining (feat. Beyoncé & Jay-Z)', 'Track_id': '7Fa5UNizycSms5jP3SQD3F', 'Prediction': 1}
{'Track Name': 'You Gotta Not', 'Track_id': '0B1QTVABE86QcHjXdPfeej', 'Prediction': 1}
{'Track Name': 'The Way Life Goes (feat. Oh Wonder)', 'Track_id': '2eAZfqOm4EnOF9VvN50Tyc', 'Prediction': 1}
{'Track Name': 'Renegades Of Funk', 'Track_id': '5YBVDvTSSSiqv7KZDeUlXA', 'Prediction': 1}
{'Track Name': 'Word Up!', 'Track_id': '4D05EjPFWFx3BEQNeB7Vct', 'Prediction': 1}
{'Track Name': 'Misery Business', 'Track_id': '6SpLc7EXZIPpy0sVko0aoU', 'Prediction': 1}
{'Track Name': 'Magnolia', 'Track_id': '1e1JKLEDKP7hEQzJfNAgPl', 'Prediction': 1}
{'Track Name': 'Over and Over Again', 'Track_id': '3nhGcnuGvsLL8OtjoOOxXp', 'Prediction': 1}
{'Track Name': 'Give It Away', 'Track_id': '2rmoeBqloVIhg7Zb20n5Sq', 'Prediction': 1}
{'Track Name': 'Talk Is Cheap', 'Track_id': '7jJsme36M90Cn6aYYC2l3t', 'Prediction': 1}
{'Track Name': 'Applause', 'Track_id': '2Zj4FUsMtu9PMuJsHbGbdv', 'Prediction': 1}
{'Track

## The recommendations are:

In [24]:
dfResults = pd.DataFrame.from_dict(dictRecommend)
dfResults

,Track Name,Track_id,Prediction
0,Shining (feat. Beyoncé & Jay-Z),7Fa5UNizycSms5jP3SQD3F,1
1,You Gotta Not,0B1QTVABE86QcHjXdPfeej,1
2,The Way Life Goes (feat. Oh Wonder),2eAZfqOm4EnOF9VvN50Tyc,1
3,Renegades Of Funk,5YBVDvTSSSiqv7KZDeUlXA,1
4,Word Up!,4D05EjPFWFx3BEQNeB7Vct,1
5,Misery Business,6SpLc7EXZIPpy0sVko0aoU,1
6,Magnolia,1e1JKLEDKP7hEQzJfNAgPl,1
7,Over and Over Again,3nhGcnuGvsLL8OtjoOOxXp,1
8,Give It Away,2rmoeBqloVIhg7Zb20n5Sq,1
9,Talk Is Cheap,7jJsme36M90Cn6aYYC2l3t,1
